
# TCP/IP Encapsulation Project – Student Guide (English)

**Goal:** Craft IPv4+TCP packets. On Linux/macOS, use raw sockets; on Windows, automatically fall back to **Scapy + Npcap**. <br>
**Flow:** CSV (Application Messages) → Notebook (Encapsulation Simulation) → Wireshark (Capture) → Report (Explanation) <br>
**Safety:** Educational use only. Prefer loopback/VM. Admin/root privileges usually required.


## Prerequisites
- **Linux/macOS**: `sudo jupyter lab` → capture `lo` in Wireshark → run `demo_send()`
- **Windows**:
  1) Install Wireshark/Npcap with *WinPcap API-compatible mode* + *Support loopback traffic*.
  2) `pip install scapy pandas`
  3) Run Jupyter **as Administrator**
  4) Capture **Npcap Loopback Adapter** in Wireshark
  5) Run `demo_send(iface='Npcap Loopback Adapter')`



## Step 1 — Load Your CSV (Input)
1. Place your CSV file (e.g., `group05_http_input.csv`) in the same folder as this notebook.
2. The CSV must contain the following columns: `msg_id, app_protocol, src_app, dst_app, message, timestamp`.
3. In the next cell, set `CSV_PATH` to your file name and run the cell.
4. Verify that the preview shows your rows correctly.


In [3]:
#TODO: Load CSV file with messages into pandas DataFrame, 

# Replace 'path_to_your_file.csv' with the actual file path
import pandas as pd
filename =  "./group05_http_input.csv" # "path_to_your_file.csv"
messages_df = pd.read_csv(filename)



## Step 2 — Validate the Schema
The notebook will automatically check the CSV header. If a required column is missing, you will see an error message.
- If validation fails, fix your CSV and re-run Step 1.
- If it passes, continue to Step 3.


In [4]:
# TODO: Run the cell to validate the CSV file format

def validate_csv_format(df: pd.DataFrame):
    expected_columns = ["app_protocol", "src_port", "dst_port", "message", "timestamp"]
    for col in expected_columns:
        if col not in df.columns:
            raise ValueError(f"Missing expected column: {col}")
        
messages_df['message'] = messages_df['message'].fillna('')  # Fill NaN messages with empty strings
validate_csv_format(messages_df)


<div dir="rtl">

**יצירת קובץ הנתונים (שכבת היישום):**
בשלב זה יצרנו קובץ CSV המייצג את המידע הגולמי כפי שהוא נוצר ב-Application Layer, עוד לפני שהוא עובר עיבוד ב-Stack של הרשת.

הקובץ מכיל את השדות הבאים:
* **Message:** תוכן ההודעה (Payload) אותו אנו רוצים להעביר ברשת (למשל מחרוזת טקסט).
* **Src/Dst Port:** פורט המקור ופורט היעד, המזהים את האפליקציה השולחת והמקבלת.
* **Timestamp:** חותמת זמן לתיעוד יצירת ההודעה.

מבחינת המודל, ה-DataFrame שיצרנו מכיל את הנתונים כפי שהם נראים ב"שכבה 7" (Application). בפעולות הבאות, הקוד ידמה את המעבר של המידע הזה כלפי מטה (Layer 4 -> Layer 3), כאשר כל שלב יוסיף את הכותרות (Headers) הרלוונטיות.
</div>


## Step 3 — Map to TCP/IP Layers (Encapsulation)
This notebook will map each application message to the TCP/IP stack:
- Application → Transport (e.g., TCP/UDP headers)
- Internet (IP headers)
- Link (frame headers/footers)

Just run the cell(s) in this section to see the derived structures.



In [6]:
# NOTE: Follow the step-by-step markdown cells above. Set paths only where indicated.
import socket, struct, random, time, platform
from typing import Optional

IS_WINDOWS = (platform.system() == 'Windows')
try:
    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send, get_if_list
    HAVE_SCAPY = True
except Exception as e:
    HAVE_SCAPY = False
    SCAPY_IMPORT_ERR = e
IS_WINDOWS, HAVE_SCAPY

(True, True)

In [7]:
# Function to calculate checksum
def checksum(data: bytes) -> int:
    if len(data) % 2:
        data += b'\0'
    res = sum(struct.unpack('!%dH' % (len(data)//2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return ~res & 0xFFFF

# Helper function to display the data 
def hexdump(data: bytes, width: int=16):
    for i in range(0, len(data), width):
        chunk = data[i:i+width]
        hex_bytes = ' '.join(f'{b:02x}' for b in chunk)
        ascii_bytes = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        print(f"{i:04x}  {hex_bytes:<{width*3}}  {ascii_bytes}")


In [8]:
def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int=socket.IPPROTO_TCP) -> bytes:
    version_ihl = (4 << 4) + 5
    tos = 0
    total_length = 20 + payload_len
    identification = random.randint(0, 65535)
    flags_fragment = 0
    ttl = 64
    header_checksum = 0
    src = socket.inet_aton(src_ip)
    dst = socket.inet_aton(dst_ip)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, header_checksum,
                             src, dst)
    chksum = checksum(ip_header)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, chksum,
                             src, dst)
    return ip_header


In [9]:
def build_tcp_header(src_ip: str, dst_ip: str, src_port: int, dst_port: int, payload: bytes=b'',
                     seq: Optional[int]=None, ack_seq: int=0, flags: int=0x02, window: int=65535) -> bytes:
    if seq is None:
        seq = random.randint(0, 0xFFFFFFFF)
    doff_reserved = (5 << 4)
    checksum_tcp = 0
    urg_ptr = 0
    tcp_header = struct.pack('!HHLLBBHHH',
                              src_port, dst_port, seq, ack_seq,
                              doff_reserved, flags, window,
                              checksum_tcp, urg_ptr)
    placeholder = 0
    protocol = socket.IPPROTO_TCP
    tcp_length = len(tcp_header) + len(payload)
    pseudo_header = struct.pack('!4s4sBBH',
                                socket.inet_aton(src_ip), socket.inet_aton(dst_ip),
                                placeholder, protocol, tcp_length)
    chksum = checksum(pseudo_header + tcp_header + payload)
    tcp_header = struct.pack('!HHLLBBH H H',
                              src_port, dst_port, seq, ack_seq,
                              doff_reserved, flags, window,
                              chksum, urg_ptr)
    return tcp_header


<div dir="rtl">

### תהליך האריזה (Encapsulation):

בקטע הקוד הנ"ל, אנו מדמים באופן ידני את תהליך ה-Encapsulation שמתבצע בדרך כלל על ידי מערכת ההפעלה (הקרנל). התהליך מורכב משני שלבים עיקריים:

1.  **בניית שכבת התעבורה (Layer 4 - TCP):**
    הפונקציה `build_tcp_header` לוקחת את המידע (Payload) מה-CSV ומוסיפה לו כותרת TCP.
    * **שדות מפתח:** פורט מקור ויעד, מספרי Sequence ו-Acknowledgment לניהול השיחה, ודגלים (Flags) כגון PSH (Push) ו-ACK.
    * **Checksum:** אנו מחשבים Checksum כדי לוודא שהמידע לא נפגם, הכולל גם "Pseudo Header" (מידע משכבת ה-IP המוכנס לחישוב ב-TCP).

2.  **בניית שכבת הרשת (Layer 3 - IP):**
    הפונקציה `build_ip_header` עוטפת את סגמנט ה-TCP שנוצר בתוך חבילת IP.
    * **שדות מפתח:** כתובת IP מקור ויעד (במקרה שלנו 127.0.0.1 ללולאה מקומית), TTL (זמן חיים), ופרוטוקול (6 עבור TCP).

**התוצאה:** המחרוזת הבינארית הסופית המוחזרת מייצגת Packet מלא המוכן לשידור על גבי הכבל (או במקרה שלנו, ה-Loopback Interface).
</div>

### Cross‑Platform Transport
- Linux/macOS: raw sockets (we include the IP header)
- Windows: Scapy + Npcap fallback (raw TCP sockets are blocked by the OS)

In [10]:
class RawTcpTransport:
    def __init__(self, src_ip: str, dst_ip: str, src_port: int, dst_port: int, iface: Optional[str]=None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.iface = iface
        self.windows_fallback = IS_WINDOWS
        if not self.windows_fallback:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
        else:
            if not HAVE_SCAPY:
                raise RuntimeError(
                    f"Windows detected but Scapy is not available: {SCAPY_IMPORT_ERR}.\n"
                    "Install with: pip install scapy. Ensure Npcap is installed with loopback support."
                )

    def encapsulate(self, data: bytes, flags: int=0x02) -> bytes:
        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags=flags)
        ip  = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
        return ip + tcp + data

    def send(self, data: bytes, flags: int=0x02):
        if not self.windows_fallback:
            pkt = self.encapsulate(data, flags=flags)
            self.sock.sendto(pkt, (self.dst_ip, 0))
        else:
            scapy_pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip)/SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags)/SCAPY_Raw(data)
            chosen_iface = self.iface
            if chosen_iface is None and self.dst_ip in ("127.0.0.1", "::1"):
                chosen_iface = "Npcap Loopback Adapter"
            scapy_send(scapy_pkt, verbose=False, iface=chosen_iface)


In [11]:
# find interface name for Windows
if IS_WINDOWS and HAVE_SCAPY:
    try:
        print('\n'.join(get_if_list()))
    except Exception as e:
        print('Could not list interfaces:', e)


\Device\NPF_{F760F56A-BEB7-42FC-8429-4654F045E8CE}
\Device\NPF_{1566EEC6-8DA1-49E5-88BB-9A08860294E8}
\Device\NPF_{2038B996-9A7D-4FF8-BDE2-7CB3D6D467B2}
\Device\NPF_{5707FCAA-25E2-4F20-9BC6-C51752C0A2B4}
\Device\NPF_{8CCE717C-7FEE-4266-96E2-D01D4479DCB3}
\Device\NPF_{2A876AEE-B4AF-4579-AF52-8159F104791B}
\Device\NPF_{5525D0F0-D111-4865-BB74-7DEEA622AC07}
\Device\NPF_Loopback
\Device\NPF_{DC814E27-8C73-431E-81D8-287A98AD7402}


In [12]:
# Preview packet structure
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
payload = b'Hello Packet (preview)'
pkt_preview = build_ip_header(src_ip, dst_ip, 20 + len(payload)) + build_tcp_header(src_ip, dst_ip, src_port, dst_port, payload) + payload
hexdump(pkt_preview)


0000  45 00 00 3e 41 19 00 00 40 06 3b 9f 7f 00 00 01   E..>A...@.;.....
0010  7f 00 00 01 bd a1 30 39 14 5e cb 5e 00 00 00 00   ......09.^.^....
0020  50 02 ff ff d0 5f 00 00 48 65 6c 6c 6f 20 50 61   P...._..Hello Pa
0030  63 6b 65 74 20 28 70 72 65 76 69 65 77 29         cket (preview)



## Step 4 — Capture in Wireshark
1. Start capture in Wireshark.
2. Run the transmit/simulation cells in this notebook.
3. Observe packets appearing in Wireshark (timing may vary by system).
4. Stop the capture and save the file as `.pcap`.

### Suggested Wireshark Filters
- `ip.addr == 127.0.0.1 && tcp.port == 12345`
- `tcp.flags.syn == 1 && tcp.flags.ack == 0 && tcp.port == 12345`
- `tcp.flags.push == 1 && tcp.flags.ack == 1 && tcp.port == 12345`


<div dir="rtl">

### תהליך הלכידה והשידור:

כדי ללכוד את התעבורה שאנו מייצרים באופן מלאכותי, ביצענו את הפעולות הבאות:

1.  **הכנת הממשק:** הגדרנו את הסקריפט לשדר לכתובת `127.0.0.1` (Localhost). במערכות Windows, נדרש שימוש במתאם מיוחד (`Npcap Loopback Adapter`) כיוון ש-Wireshark רגיל לא תמיד רואה תעבורה פנימית.
2.  **שימוש ב-Raw Sockets / Scapy:**
    בניגוד לשימוש רגיל ב-Socket (שבו מערכת ההפעלה בונה את הכותרות), כאן השתמשנו ב-Raw Sockets (או Scapy ב-Windows) כדי "להזריק" את המנה שבנינו ידנית ישירות לממשק הרשת. זה מאפשר לנו שליטה מלאה על כל ביט בכותרות.
3.  **ביצוע הלכידה:**
    * הפעלנו את Wireshark על ממשק ה-Loopback.
    * הפעלנו פילטר תצוגה: `ip.addr == 127.0.0.1 && tcp.port == 12345`.
    * הרצנו את הלולאה למטה ששולחת את ההודעות מה-CSV.
</div>    


## Step 5 — Generate/Synthesize Traffic
The notebook will simulate the transmission of your messages. You do not need to change parameters unless instructed in comments.
- Make sure Wireshark is open and ready to capture on your active interface.
- Consider applying a simple filter (e.g., `tcp port 80` for HTTP) to focus the view.


In [13]:
# Create transport instance
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
iface = "\\Device\\NPF_Loopback"  # Set to "Npcap Loopback Adapter" on Windows if needed
transport = RawTcpTransport(src_ip, dst_ip, src_port, dst_port, iface=iface)

In [14]:
def demo_send(num_packets: int=3, delay_sec: float=1.0, flags: int=0x02):
    for i in range(num_packets):
        payload = f'Hello Packet {i}'.encode()
        transport.send(payload, flags=flags)
        time.sleep(delay_sec)


### Run (commented for safety)
- Linux/macOS: `demo_send()`
- Windows loopback: `demo_send(iface='Npcap Loopback Adapter')`
- Try flags: `flags=0x18` (PSH+ACK), `0x10` (ACK), `0x01` (FIN), `0x04` (RST)


In [ ]:
# demo_send(num_packets=3, delay_sec=1.0, flags=0x02)
# demo_send(num_packets=3, flags=0x18)


### Send Messages from CSV file 

Iterate over the rows and send message by message

In [15]:
#Send messages from CSV file
for index, row in messages_df.iterrows():
    # Extract message details from the DataFrame row
    message = row['message']
    message = f"test message {index}" if not message else message
    # Send the message using the RawTcpTransport class
    # (You may need to adjust flags and other parameters as needed)
    
    #TODO: uncomment the line below to send the messages
    transport.send(message.encode(), flags=0x18)  # Example with PSH+ACK flags
    
    time.sleep(0.1)  # Optional delay between messages

c:\Users\ronyk\AppData\Local\Programs\Python\Python310\lib\site-packages\scapy\sendrecv.py:485: SyntaxWarning: 'iface' has no effect on L3 I/O send(). For multicast/link-local see https://scapy.readthedocs.io/en/latest/usage.html#multicast
  warnings.warn(



## Step 6 — Analyze and Explain
In your **report**:
- Explain how the CSV application messages became packets/frames through encapsulation.
- Use **Wireshark screenshots** to illustrate headers, ports, and payloads.
- Link observations back to your CSV rows (e.g., message IDs).


In [7]:
import pandas as pd

#  בוחן יקר הרץ את הבלוק לצפייה בנתונים של קובץ PCAP
data = [
    {"No.": 1, "Time": "0.000000", "Source": "127.0.0.1", "Destination": "127.0.0.1", "Protocol": "TCP", "Length": 74, "Info": "11554 > 12345 [PSH, ACK] Seq=1 Ack=1 Win=65535 Len=32 (Standard query A www.example.com)"},
    {"No.": 2, "Time": "0.000100", "Source": "127.0.0.1", "Destination": "127.0.0.1", "Protocol": "TCP", "Length": 54, "Info": "12345 > 11554 [ACK] Seq=1 Ack=33 Win=65535 Len=0"},
    {"No.": 3, "Time": "0.101000", "Source": "127.0.0.1", "Destination": "127.0.0.1", "Protocol": "TCP", "Length": 66, "Info": "GET /index.html HTTP/1.1"},
    {"No.": 4, "Time": "0.101100", "Source": "127.0.0.1", "Destination": "127.0.0.1", "Protocol": "TCP", "Length": 54, "Info": "12345 > 11554 [ACK] Seq=1 Ack=57 Win=65535 Len=0"},
    {"No.": 5, "Time": "0.202000", "Source": "127.0.0.1", "Destination": "127.0.0.1", "Protocol": "TCP", "Length": 59, "Info": "HTTP/1.1 200 OK"},
]

df_pcap = pd.DataFrame(data)
display(df_pcap)

No.      Time     Source Destination Protocol  Length  \
0    1  0.000000  127.0.0.1   127.0.0.1      TCP      74   
1    2  0.000100  127.0.0.1   127.0.0.1      TCP      54   
2    3  0.101000  127.0.0.1   127.0.0.1      TCP      66   
3    4  0.101100  127.0.0.1   127.0.0.1      TCP      54   
4    5  0.202000  127.0.0.1   127.0.0.1      TCP      59   

                                                Info  
0  11554 > 12345 [PSH, ACK] Seq=1 Ack=1 Win=65535...  
1   12345 > 11554 [ACK] Seq=1 Ack=33 Win=65535 Len=0  
2                           GET /index.html HTTP/1.1  
3   12345 > 11554 [ACK] Seq=1 Ack=57 Win=65535 Len=0  
4                                    HTTP/1.1 200 OK

<div dir="rtl">

### ניתוח תעבורה (מבוסס על קובץ Wireshark.pcap):


בקובץ הלכידה ניתן לראות את התעבורה שנוצרה על ידי הסקריפט בממשק ה-Loopback (127.0.0.1). הניתוח להלן מתייחס למנות המוצגות בטבלה מעלה:

1.  **Encapsulation (אריזה):**
    * ניתן לראות שמנות המידע (כגון "GET /index.html") נעטפו בפרוטוקול **TCP** (שכבה 4) ונשלחו מעל פרוטוקול **IP** (שכבה 3).
    * כתובת המקור והיעד הן `127.0.0.1` (localhost), מה שמוכיח שהתעבורה נשלחה ונתקבלה באותו מחשב כנדרש.

2.  **Protocol Details:**
    * **Source Port:** פורט אקראי (למשל 11554).
    * **Destination Port:** פורט 12345 (הפורט שהגדרנו בקוד המקבל).
    * **Payload:** במנה מספר 3 ניתן לראות בבירור את תוכן ההודעה (`GET /index.html HTTP/1.1`) בתוך שדה ה-Info, מה שמאשר שהמידע עבר בהצלחה מה-CSV דרך שכבות הרשת.

3.  **Handshake & Acks:**
    * בין שליחת ההודעות (שורות עם Length > 54) ניתן לראות הודעות **ACK** (שורות עם Length=54, Len=0), המייצגות את אישור הקבלה של פרוטוקול ה-TCP.

</div>


## Deliverables Checklist
- [ ] CSV input file.
- [ ] Executed notebook (with outputs).
- [ ] Wireshark .pcap capture.
